In [1]:
%run task_setup.ipynb

Foundation: 2.11.030
Transition: 3.02.029


-------------------------
## Transition

In [2]:
for task in ['members', 'cln_members', 'ins_members', 'prf_members', 'flu_risk']:
    tr = Transition.from_env(task)
    tr.pm.reset_intents()
    tr.pm_persist()
    df = tr.load_source_canonical()
    
    # code clean
    tr.cleaners.auto_clean_header(df, inplace=True)
    tr.cleaners.auto_transition(df, inplace=True)
    if task == 'members':
        tr.cleaners.to_remove(df, headers=['email', 'state_abbr', 'address'], inplace=True)
        tr.cleaners.to_str_type(df, headers='zipcode', fixed_len_pad='0', use_string_type=True, inplace=True)
    
    # run pipeline
    tr.run_component_pipeline()


In [3]:
tr = Transition.from_env('members')
tr.canonical_report(tr.load_persist_canonical())

,Attributes (16),dType,%_Null,%_Dom,Count,Unique,Observations
0,age,int64,0.0%,3.3%,100000,71,max=91 | min=21 | mean=39.46 | dominant=26
1,city,string,0.0%,1.9%,100000,7583,Sample: New York | Los Angeles | Sacramento | San Diego | Columbus
2,county,string,0.0%,3.7%,100000,770,Sample: Los Angeles | Jefferson | New York | Suffolk | Clark
3,county_fips,int64,0.0%,3.7%,100000,1074,max=55141 | min=6001 | mean=26483.03 | dominant=6037
4,family_name,string,0.0%,0.0%,100000,73181,Sample: Sendler | Sporbert | Trau | Cucalon | Aynes
5,first_name,string,0.0%,0.0%,100000,34561,Sample: Kamaree | Lacy | Maurice | Yves | Chesley
6,gender,category,0.0%,55.0%,100000,2,Sample: F | M
7,is_PCP_auto_assigned,bool,0.0%,67.5%,100000,2,True | False
8,line_of_business,category,0.0%,76.0%,100000,2,Sample: Commercial | Business
9,member_health_plan,category,0.0%,60.0%,100000,2,Sample: PPO | HMO


-------------------------
## Wrangle

In [4]:
wr = Wrangle.from_env('profile')
tools = wr.tools

In [5]:
wr.pm.reset_intents()
wr.pm_persist()

#### Code Edit

In [6]:
# get the source
df = wr.load_source_canonical()

In [7]:
# role through the other connectors and merge
for task in ['cln_members', 'ins_members', 'prf_members', 'flu_risk']:
    df = tools.model_merge(df, other=task, on='member_id', how='left', column_name=f'{task}_merge')
    wr.add_column_description(column_name=f'{task}_merge', description=f"The merge of the {task} dataset into the profile on member_id")
    

#### Run Pipeline

In [8]:
wr.run_component_pipeline()

In [9]:
wr.canonical_report(wr.load_persist_canonical())

,Attributes (35),dType,%_Null,%_Dom,Count,Unique,Observations
0,age,int64,0.0%,3.3%,100000,71,max=91 | min=21 | mean=39.46 | dominant=26
1,city,string,0.0%,1.9%,100000,7583,Sample: New York | Los Angeles | Sacramento | San Diego | Columbus
2,cln_aptmt_dt,datetime64[ns],0.0%,0.7%,100000,366,max=2021-02-20 00:00:00 | min=2020-02-21 00:00:00 | yr mean= 2020
3,cln_bmi,float64,0.0%,2.9%,100000,672,max=67.0 | min=0.0 | mean=32.15 | dominant=19.6
4,cln_diabetes,bool,0.0%,60.3%,100000,2,False | True
5,cln_diastolic,int64,0.0%,3.0%,100000,122,max=121 | min=0 | mean=74.82 | dominant=0
6,cln_dpf,float64,0.0%,2.9%,100000,2193,max=2.419 | min=0.078 | mean=0.61 | dominant=0.832
7,cln_glucose,int64,0.0%,3.6%,100000,199,max=198 | min=0 | mean=126.21 | dominant=119
8,cln_insulin,float64,0.0%,54.0%,100000,835,max=845.0 | min=0.0 | mean=87.72 | dominant=0.0
9,cln_pregnancies,int64,0.0%,50.8%,100000,17,max=16 | min=0 | mean=2.66 | dominant=0


-------------------------
## Transition

In [10]:
tr = Transition.from_env('profile')

In [11]:
tr.pm.reset_intents()
tr.pm_persist()

In [12]:
df = tr.load_source_canonical()

#### Run Pipeline

In [13]:
tr.run_component_pipeline()

In [14]:
tr.canonical_report(tr.load_persist_canonical())

,Attributes (35),dType,%_Null,%_Dom,Count,Unique,Observations
0,age,int64,0.0%,3.3%,100000,71,max=91 | min=21 | mean=39.46 | dominant=26
1,city,string,0.0%,1.9%,100000,7583,Sample: New York | Los Angeles | Sacramento | San Diego | Columbus
2,cln_aptmt_dt,datetime64[ns],0.0%,0.7%,100000,366,max=2021-02-20 00:00:00 | min=2020-02-21 00:00:00 | yr mean= 2020
3,cln_bmi,float64,0.0%,2.9%,100000,672,max=67.0 | min=0.0 | mean=32.15 | dominant=19.6
4,cln_diabetes,bool,0.0%,60.3%,100000,2,False | True
5,cln_diastolic,int64,0.0%,3.0%,100000,122,max=121 | min=0 | mean=74.82 | dominant=0
6,cln_dpf,float64,0.0%,2.9%,100000,2193,max=2.419 | min=0.078 | mean=0.61 | dominant=0.832
7,cln_glucose,int64,0.0%,3.6%,100000,199,max=198 | min=0 | mean=126.21 | dominant=119
8,cln_insulin,float64,0.0%,54.0%,100000,835,max=845.0 | min=0.0 | mean=87.72 | dominant=0.0
9,cln_pregnancies,int64,0.0%,50.8%,100000,17,max=16 | min=0 | mean=2.66 | dominant=0
